In [ ]:
!pip install -qU \
  langchain-core  \
  langchain[google-genai]  \
  langchain-community  \
  langsmith  \
  google-search-results 

## Tools

In [ ]:
from langchain_core.tools import tool

@tool
def add(x: float, y:float) -> float:
    """Add 'x' and 'y'."""
    return x + y

@tool
def subtract(x: float, y:float) -> float:
    """Subtract 'x' and 'y'."""
    return x - y

@tool
def multiply(x: float, y:float) -> float:
    """Multiply 'x' and 'y'."""
    return x * y

@tool
def divide(x: float, y:float) -> float:
    """Divide 'x' and 'y'."""
    return x / y

@tool
def exponentiate(x: float, y: float) -> float:
    """Raise 'x' to the power of 'y'."""
    return x ** y



In [ ]:
add

In [ ]:
print(f"{add.name}\n{add.description}\n{add.args_schema.model_json_schema()}")

In [ ]:
exponentiate.args_schema.model_json_schema()

In [ ]:
import json

llm_output_string = "{\"x\": 5, \"y\": 2}"
llm_output_dict = json.loads(llm_output_string)
llm_output_dict

In [ ]:
exponentiate.func(**llm_output_dict)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful assistant"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}")
])

In [ ]:
import getpass
import os
from langchain_google_genai import GoogleGenerativeAI

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

#llm = GoogleGenerativeAI(model="gemini-2.0-flash", temperature=0.0) 


from langchain.chat_models import init_chat_model

llm = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

In [ ]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    memory_key = "chat_history",
    return_messages = True
)

In [ ]:
from langchain.agents import create_tool_calling_agent

tools = [add, subtract, multiply, divide, exponentiate]

agent = create_tool_calling_agent(
    llm=llm, tools=tools, prompt=prompt
)

In [ ]:
agent.invoke({
    "input": "What is 10.7 multiplied by 7.68",
    "chat_history": memory.chat_memory.messages,
    "intermediate_steps":[]
})


In [ ]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(
    agent = agent,
    tools = tools,
    memory = memory,
    verbose = True
)

In [ ]:
agent_executor.invoke({
    "input": "What is 10.7 multiplied by 7.68",
    "chat_history": memory.chat_memory.messages
})

In [ ]:
memory.chat_memory.messages

In [ ]:
agent_executor.invoke({
    "input": "My name is Muhammad",
    "chat_history": memory.chat_memory.messages
})

In [ ]:
agent_executor.invoke({
    "input": "What is nine plus 10, minus 4 * 2 to the power of 3",
    "chat_history": memory.chat_memory.messages
})

In [ ]:
agent_executor.invoke({
    "input": "9+10-4*2**3",
    "chat_history": memory.chat_memory.messages
})

In [ ]:
agent_executor.invoke({
    "input": "What is my name",
    "chat_history": memory.chat_memory.messages
})

### API Calling

In [ ]:
import requests
from datetime import datetime

@tool
def get_location_from_ip():
    """Get Geographical Location from IP"""
    
    try:
        response = requests.get("https://ipinfo.io/json")
        
        data = response.json()
        if "loc" in data:
            latitude, longitude = data["loc"].split(",")
        
        data = (
                f"Latitude: {latitude},\n"
                f"Longitude: {longitude},\n"
                f"City: {data.get('city', 'N/A')},\n"
                f"Country: {data.get('country', 'N/A')}"
        )
        return data
    except Exception as e:
        return f"Error Occured {e}"

@tool
def get_current_datetime():
    """Return the current date and time."""
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

In [ ]:
if not os.environ.get("SERPAPI_API_KEY"):
  os.environ["SERPAPI_API_KEY"] = getpass.getpass("Enter your SerpAPI API key: ")


In [ ]:
print(f"{get_location_from_ip.name}\n{get_location_from_ip.description}\n{get_location_from_ip.args_schema.model_json_schema()}")
print(f"{get_current_datetime.name}\n{get_current_datetime.description}\n{get_current_datetime.args_schema.model_json_schema()}")
 

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You're a helpful assistance"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}")
])

In [ ]:
from langchain.agents import load_tools

toolbox = load_tools(tool_names=['serpapi'], llm=llm)

tools = toolbox + [get_location_from_ip, get_current_datetime]

agent = create_tool_calling_agent(
    llm = llm, tools = tools, prompt = prompt
)

agent_executor = AgentExecutor(
    agent = agent,
    tools = tools, 
    verbose = True
)



In [ ]:
agent_executor.invoke({
    "input": (
        "I have a few questions, what is the date and time right now? "
        "How is the weather where I am? Please give me degrees in Celsius"
    )
})